In [1]:
import os
os.chdir( "../")

In [2]:
import psutil
num_cpus = psutil.cpu_count(logical=False)
num_cpus 


8

In [3]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [4]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [5]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seeds = [
    1811854, 3772146, 6407100, 6926227, 1340378, 
    68679, 3822303, 4058970, 1264160, 1100338, 
    6552126, 725363, 8362134, 9446122, 5256199, 
    4686339, 3830906, 7315366, 9428914, 5564236, 
    3420043, 9223036, 851434, 3300082, 7899660, 
    2053768, 4305274, 3709256, 8795594, 1418903, 
    7428566, 157832
]

seeds = seeds[:num_cpus]

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [7]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [8]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2166 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2150 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2190 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2194 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2165 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2145 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2171 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2198 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [9]:
import os

output_folder_path = "result_experiments_xg/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 1000  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [10]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative, runHyperparameterSearch_Content
from Recommenders.NonPersonalizedRecommender import TopPop, Random
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
import os, multiprocessing
from functools import partial
from skopt.space import Real, Integer, Categorical


2024-01-07 21:05:24.641327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-07 21:05:24.641362: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
from concurrent.futures import ProcessPoolExecutor
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
import time
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender

In [12]:
import optuna

host = "192.168.16.83"
port = 3306
database_name = "optuna_mio"
username = "optuna"
password = "your_password"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url
storage = optuna.storages.RDBStorage(url=mysql_url)

study_name = "IALS_8FOLD"

study = optuna.create_study(study_name=study_name, storage=storage, load_if_exists=True, direction="maximize", sampler=optuna.samplers.CmaEsSampler())

model = ImplicitALSRecommender

[I 2024-01-07 21:05:26,185] A new study created in RDB with name: IALS_8FOLD


In [13]:
def objective(trial):

    params = {
        "num_factors": trial.suggest_int("num_factors",1, 1500),
        "epochs": trial.suggest_int("epochs",low = 50, high = 2500),
        "confidence_scaling": trial.suggest_categorical("confidence_scaling",["linear", "log"]),
        "alpha": trial.suggest_float("alpha",low = 1e-3, high = 50, log=True),
        "epsilon": trial.suggest_float("epsilon",low = 1e-3, high = 10, log=True),
        "reg": trial.suggest_float("reg",low = 1e-5, high = 1e-2, log=True),
    }


    score = 0
    for i, _ in enumerate(seeds):
        recommender = model(urm_train_validation[i])
        recommender.fit(use_gpu=True, num_threads=1, **params)
        
        result, _ = evaluator_test[i].evaluateRecommender(recommender)
        MAP_result = result["MAP"].item()
            
        score += MAP_result
    
    score = score /len(seeds)
       
    return score

In [14]:
study.optimize(objective, n_trials=200)

ImplicitALSRecommender: URM Detected 229 ( 1.8%) users with no interactions.
ImplicitALSRecommender: URM Detected 126 ( 0.6%) items with no interactions.
Using gpu: True


  0%|          | 0/987 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 10472 (100.0%) in 4.33 sec. Users per second: 2419
ImplicitALSRecommender: URM Detected 217 ( 1.7%) users with no interactions.
ImplicitALSRecommender: URM Detected 113 ( 0.5%) items with no interactions.
Using gpu: True


  0%|          | 0/987 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 10488 (100.0%) in 5.45 sec. Users per second: 1924
ImplicitALSRecommender: URM Detected 223 ( 1.8%) users with no interactions.
ImplicitALSRecommender: URM Detected 106 ( 0.5%) items with no interactions.
Using gpu: True


...

  0%|          | 0/1358 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 10472 (100.0%) in 6.67 sec. Users per second: 1569
ImplicitALSRecommender: URM Detected 217 ( 1.7%) users with no interactions.
ImplicitALSRecommender: URM Detected 113 ( 0.5%) items with no interactions.
Using gpu: True


  0%|          | 0/1358 [00:00<?, ?it/s]

[W 2024-01-08 00:16:05,250] Trial 14 failed with parameters: {'num_factors': 793, 'l1_ratio': 1358, 'confidence_scaling': 'linear', 'alpha': 6.823032501465704, 'epsilon': 0.015378609491421377, 'reg': 0.000799877970842227} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_137949/2513788561.py", line 16, in objective
    recommender.fit(use_gpu=True, num_threads=1, **params)
  File "/home/japo/RecSys-Challenge/Recommenders/MatrixFactorization/ImplicitIALSRecommender.py", line 57, in fit
    self.rec.fit(conf_scale(self.URM_train, alpha, epsilon), show_progress=self.verbose)
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/implicit/gpu/als.py", line 151, in fit
    self.solver.least_squares(Cui, X, self._YtY, Y, self.cg_steps)
KeyboardIn

KeyboardInterrupt: 